## Books Database Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
!mongoimport --type csv -d books_db -c books --headerline --drop Resources/Books.csv
!mongoimport --type csv -d books_db -c users --headerline --drop Resources/Users.csv
!mongoimport --type csv -d books_db -c ratings --headerline --drop Resources/Ratings.csv
!mongoimport --type csv -d books_db -c geodata --headerline --drop Resources/GeoLoc.csv

2023-12-10T21:34:09.464-0500	connected to: mongodb://localhost/
2023-12-10T21:34:09.464-0500	dropping: books_db.books
2023-12-10T21:34:09.622-0500	7253 document(s) imported successfully. 0 document(s) failed to import.
2023-12-10T21:34:09.750-0500	connected to: mongodb://localhost/
2023-12-10T21:34:09.750-0500	dropping: books_db.users
2023-12-10T21:34:09.814-0500	2517 document(s) imported successfully. 0 document(s) failed to import.
2023-12-10T21:34:09.972-0500	connected to: mongodb://localhost/
2023-12-10T21:34:09.972-0500	dropping: books_db.ratings
2023-12-10T21:34:10.116-0500	6560 document(s) imported successfully. 0 document(s) failed to import.
2023-12-10T21:34:10.271-0500	connected to: mongodb://localhost/
2023-12-10T21:34:10.272-0500	dropping: books_db.geodata
2023-12-10T21:34:10.313-0500	93 document(s) imported successfully. 0 document(s) failed to import.


In [4]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'books_db', 'config', 'local', 'met', 'uk_food']

In [5]:
# assign the uk_food database to a variable name
db = mongo['books_db']

In [6]:
collections_to_drop_list=['merged','merged_ratings_users','merged_users_geo']
for i in collections_to_drop_list:
    if i in db.list_collection_names():
        db[i].drop()

In [7]:
# review the collections in our new database
db.list_collection_names()

['users', 'books', 'ratings', 'geodata']

In [8]:
# assign the collection to a variable
books=db['books']
ratings=db['ratings']
users=db['users']
geodata=db['geodata']

In [9]:
print("Example of a record within 'books' collection: ")
pprint(books.find_one({}))
print("\n Example of a record within 'ratings' collection:")
pprint(ratings.find_one({}))
print("\n Example of a record within 'users' collection:")
pprint(users.find_one({}))
print("\n Example of a record within 'geodata' collection:")
pprint(geodata.find_one({}))

Example of a record within 'books' collection: 
{'': 14,
 'Book-Author': 'Jack Canfield',
 'Book-Title': "A Second Chicken Soup For The Woman'S Soul (Chicken Soup For "
               'The Soul Series)',
 'ISBN': 1558746218,
 'Publisher': 'Health Communications',
 'Year-Of-Publication': 1998,
 '_id': ObjectId('65767521533b4505290bcf27')}

 Example of a record within 'ratings' collection:
{'': 74,
 'Book-Rating': 10,
 'ISBN': 3125785006,
 'User-ID': 276835,
 '_id': ObjectId('65767521802702cb3e53e6b0')}

 Example of a record within 'users' collection:
{'': 252,
 'Location': 'united states',
 'User-ID': 256,
 '_id': ObjectId('657675212cb9069dac1ac441')}

 Example of a record within 'geodata' collection:
{'': 8,
 'Lat': 47.7563917,
 'Long': 7.0194428,
 '_id': ObjectId('65767522d5a4622296f57e19'),
 'country': 'australia'}


In [10]:
# merge all the data
pipeline = [{
    '$lookup': {
      'from': 'geodata',
      'localField': 'Location',
      'foreignField': 'country',
      'as': 'geo_data'
    }},
    {
    '$project': {
      '_id': 0,
      'Age': 1,
      'Location': 1,
      'User-ID': 1,
      'Geo-Data': { '$arrayElemAt': ['$geo_data', 0]}
    }}]

merged_users_geo_cursor=users.aggregate(pipeline)

for i in merged_users_geo_cursor:
    db['merged_users_geo'].insert_one(i)

pipeline = [{
    '$lookup': {
      'from': 'merged_users_geo',
      'localField': 'User-ID',
      'foreignField': 'User-ID',
      'as': 'user_data'
    }},
    {
    '$project': {
      '_id': 0,
      'Book-Rating': 1,
      'ISBN': 1,
      'User-ID': 1,
      'User':  { '$arrayElemAt': ['$user_data', 0] }
    }}]

merged_ratings_users_cursor=ratings.aggregate(pipeline)

for i in merged_ratings_users_cursor:
    db['merged_ratings_users'].insert_one(i)
    
pipeline = [{
    '$lookup': {
      'from': 'merged_ratings_users',
      'localField': 'ISBN',
      'foreignField': 'ISBN',
      'as': 'book_ratings_users'
    }
  },
  {
    '$project': {
      '_id': 0,
      'Book-Author': 1,
      'Book-Title': 1,
      'ISBN': 1,
      'Publisher': 1,
      'Year-Of-Publication': 1,
      'Ratings': "$book_ratings_users"
    }
  },
    {'$unset':['Ratings._id', 'Ratings.User._id', 'Ratings.User.Geo-Data._id']}
]
merged_cursor=books.aggregate(pipeline)

for i in merged_cursor:
    db['merged'].insert_one(i)
merged=db['merged']
    

In [11]:
print("Example of a record within 'merged' collection:")
pprint(merged.find_one({}))

Example of a record within 'merged' collection:
{'Book-Author': 'Jack Canfield',
 'Book-Title': "A Second Chicken Soup For The Woman'S Soul (Chicken Soup For "
               'The Soul Series)',
 'ISBN': 1558746218,
 'Publisher': 'Health Communications',
 'Ratings': [{'Book-Rating': 7,
              'ISBN': 1558746218,
              'User': {'Geo-Data': {'': 30,
                                    'Lat': -22.1822689,
                                    'Long': -44.68776,
                                    'country': 'canada'},
                       'Location': 'canada',
                       'User-ID': 61501},
              'User-ID': 61501},
             {'Book-Rating': 10,
              'ISBN': 1558746218,
              'User': {'Geo-Data': {'': 186,
                                    'Lat': 49.1009451,
                                    'Long': 1.9672466,
                                    'country': 'united states'},
                       'Location': 'united states',
       

In [12]:
db.list_collection_names()

['merged_ratings_users',
 'merged_users_geo',
 'users',
 'books',
 'ratings',
 'geodata',
 'merged']